### SageMaker fine tune ChatGLM

#### 准备
1. 升级boto3, sagemaker python sdk  
2. 准备requirements.txt
3. 准备s5cmd utility
4. 下载HF model，并上传到S3(optional)

In [ ]:
!pip install --upgrade boto3
!pip install --upgrade sagemaker

In [ ]:
#print('s3://{}/llm/models/'.format(sagemaker_session.default_bucket()))
#!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/
!curl -L https://github.com/peak/s5cmd/releases/download/v2.0.0/s5cmd_2.0.0_Linux-64bit.tar.gz | tar -xz && mv s5cmd ChatGLM-6B/ptuning/

In [62]:
import boto3
import sagemaker

account_id = boto3.client('sts').get_caller_identity().get('Account')
region_name = boto3.session.Session().region_name

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()

print(role)
print(bucket)

arn:aws:iam::687912291502:role/service-role/AmazonSageMaker-ExecutionRole-20211013T113123
sagemaker-us-west-2-687912291502


In [54]:
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "THUDM/chatglm-6b"#

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    #allow_patterns=allow_patterns,
)

In [87]:
print(model_download_path)
model_uri="s3://"+bucket+"/llm/model/chatglm/orignal/"
print(model_uri)

model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813
s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/


In [88]:
#using s5cmd util to fast upload model to s3
! ./ChatGLM-6B/ptuning/s5cmd sync ./model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813/ s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/
#!rm -rf model

cp model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813/configuration_chatglm.py s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/configuration_chatglm.py
cp model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813/LICENSE s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/LICENSE
cp model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813/tokenizer_config.json s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/tokenizer_config.json
cp model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813/config.json s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/config.json
cp model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e99ac3adff58b4d0b813/test_modeling_chatglm.py s3://sagemaker-us-west-2-687912291502/llm/model/chatglm/orignal/test_modeling_chatglm.py
cp model/models--THUDM--chatglm-6b/snapshots/658202d88ac4bb782b99e

### chatglm 官方P-tuning v2方式（单机单卡）
1:安装依赖lib   
2:准备数据集(本例以ADGEN 文本生成数据集为例，将解压后的 AdvertiseGen 目录放到本目录  
3:修改并bash运行 train.sh  

In [97]:
#!pip install rouge_chinese nltk jieba datasets
!git clone https://github.com/THUDM/ChatGLM-6B.git ./tmp/
#!pip install -r ChatGLM-6B/requirements.txt
!cp ChatGLM-6B/requirements.txt ChatGLM-6B/ptuning/

Cloning into './tmp'...
remote: Enumerating objects: 1094, done.
remote: Counting objects: 100% (530/530), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 1094 (delta 478), reused 473 (delta 457), pack-reused 564
Receiving objects: 100% (1094/1094), 7.91 MiB | 14.64 MiB/s, done.
Resolving deltas: 100% (647/647), done.


In [ ]:
#!cd ChatGLM-6B/ptuning/ && wget "https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1"
#!cd ChatGLM-6B/ptuning/ && mv "index.html?dl=1" dataset.tar.gz
#!cd ChatGLM-6B/ptuning/ && tar -xvf dataset.tar.gz
!./ChatGLM-6B/ptuning/s5cmd sync ChatGLM-6B/ptuning/AdvertiseGen/ s3://{bucket}/llm/chatglm/datasets/ 
#!rm -rf cd ChatGLM-6B/ptuning/dataset.tar.gz


In [100]:
# define Training Job Name 
import time
from sagemaker.huggingface import HuggingFace
instance_type="ml.g4dn.2xlarge"

job_name = f'huggingface-chatglm-simple-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/simple/'.format(sagemaker_session.default_bucket())
output_dir = '/opt/ml/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            : '50'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
from sagemaker.pytorch import PyTorch

huggingface_estimator = HuggingFace(
    entry_point          = 'start_simple.py',          # user endpoint script
    source_dir           = 'ChatGLM-6B/ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,           # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment
)

huggingface_estimator.fit(inputs=inputs)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-simple-2023-05-07-0-2023-05-07-08-58-32-616


2023-05-07 08:58:58 Starting - Starting the training job...
2023-05-07 08:59:13 Starting - Preparing the instances for training......
2023-05-07 09:00:24 Downloading - Downloading input data
2023-05-07 09:00:24 Training - Downloading the training image.....................
2023-05-07 09:03:40 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-07 09:04:03,521 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-07 09:04:03,540 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-07 09:04:03,551 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-07 09:04:03,553 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-07 09:04:07,951 sagemaker-training-toolkit INFO     Installing dependencies 

In [103]:
huggingface_estimator.model_data

's3://sagemaker-us-west-2-687912291502/huggingface-chatglm-deepspeed-2023-05-0-2023-05-07-09-30-47-212/output/model.tar.gz'

In [ ]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/huggingface-chatglm-simple-2023-05-06-1-2023-05-06-14-24-12-728/output/model.tar.gz --recursive --human-readable --summarize

In [ ]:
#For local test only
!cd ChatGLM-6B/ptuning/&& bash train.sh

### chatglm 官方deepspeed方式（全参数的Finetune,单机多卡）
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start-single-node.py,设置num-gpus  
4：触发bash ds_train_finetune_single_node.sh 

In [155]:
instance_type = 'ml.g5.48xlarge'
if instance_type in [
    "ml.p3.16xlarge",
    "ml.p3dn.24xlarge",
    "ml.g5.48xlarge",
    "ml.p4d.24xlarge"    
]:
    processes_per_host = 8
elif instance_type == "ml.p2.16xlarge":
    processes_per_host = 16
elif instance_type == "ml.g4dn.2xlarge":
    processes_per_host = 1
else:
    processes_per_host = 4
    

print("processes_per_host is set to:", processes_per_host)

processes_per_host is set to: 8


In [156]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed/'.format(sagemaker_session.default_bucket())
output_dir = '/tmp/model/adgen-chatglm-6b-ft'
#model_name_or_path = 'THUDM/chatglm-6b'
model_name_or_path = "/tmp/chatglm/"


instance_count = 1
#define the enviroment variables for your scripts.
environment = {
              
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : str(processes_per_host),
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            :'20'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start-single-node.py',          # user endpoint script
    source_dir           = 'ChatGLM-6B/ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-deepspeed-2023-05-0-2023-05-08-14-13-30-504


2023-05-08 14:13:57 Starting - Starting the training job......
2023-05-08 14:14:32 Starting - Preparing the instances for training......
2023-05-08 14:15:53 Downloading - Downloading input data
2023-05-08 14:15:53 Training - Downloading the training image..................
2023-05-08 14:18:54 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-08 14:19:50,943 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-08 14:19:51,003 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-08 14:19:51,013 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-08 14:19:51,015 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-08 14:19:55,008 sagemaker-training-toolkit INFO     Installing dependen

In [157]:
huggingface_estimator.model_data

's3://sagemaker-us-west-2-687912291502/huggingface-chatglm-deepspeed-2023-05-0-2023-05-08-14-13-30-504/output/model.tar.gz'

### chatglm deepspeed 多机多卡改造
1: 准备deepspeed lib，并修改deepspeed.json    
2：数据集（以上一致）  
3：entrypoint start.py,设置torch distribute launch configure  
4：触发bash ds_train_finetune.sh 

In [158]:
# define Training Job Name 
job_name = f'huggingface-chatglm-deepspeed-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'
#define the model s3 path which will store your trained model asset
#Note: you should use your real s3 path to configure model_s3_path
model_s3_path='s3://{}/llm/models/chatglm/deepspeed/'.format(sagemaker_session.default_bucket())
output_dir = '/tmp/model/adgen-chatglm-6b-ft'
model_name_or_path = 'THUDM/chatglm-6b'


instance_count = 2
#define the enviroment variables for your scripts.
environment = {
              'NODE_NUMBER'            : str(instance_count),
              'MODEL_S3_PATH'          : model_uri,
              'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:32',
              #'LD_LIBRARY_PATH'        : '${LD_LIBRARY_PATH}:/opt/conda/lib/',
              'NUM_GPUS'               : str(processes_per_host),
              'TRAIN_DATASET'          : '/opt/ml/input/data/AdvertiseGen/train.json',
              'TEST_DATASET'           : '/opt/ml/input/data/AdvertiseGen/dev.json',
              'PROMPT_COLUMN'          : 'content',
              'RESPONSE_COLUMN'        : 'summary',
              'MODEL_NAME_OR_PATH'     : model_name_or_path,
              'OUTPUT_DIR'             : output_dir,
              'MODEL_OUTPUT_S3_PATH'   : model_s3_path,
              'TRAIN_STEPS'            :'50'
}

inputs={
   'AdvertiseGen': f"s3://{bucket}/llm/chatglm/datasets/"
}


# create the Estimator
huggingface_estimator = HuggingFace(
    entry_point          = 'start.py',          # user endpoint script
    source_dir           = 'ChatGLM-6B/ptuning',               # directory which includes all the files needed for training
    instance_type        = instance_type, # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    script_mode          = True,
    transformers_version = '4.26',            # the transformers version used in the training job
    pytorch_version      = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    environment = environment,
)
huggingface_estimator.fit(inputs=inputs)

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-chatglm-deepspeed-2023-05-0-2023-05-08-14-36-31-547


2023-05-08 14:36:57 Starting - Starting the training job......
2023-05-08 14:37:32 Starting - Preparing the instances for training......
2023-05-08 14:38:44 Downloading - Downloading input data...
2023-05-08 14:38:59 Training - Downloading the training image...............
2023-05-08 14:41:55 Training - Training image download completed. Training in progress.......bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-05-08 14:42:42,763 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-05-08 14:42:42,825 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-05-08 14:42:42,836 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-05-08 14:42:42,838 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-05-08 14:42:46,947 sagemaker-training-toolkit INFO     Installing dependen

In [160]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/models/chatglm/deepspeed/adgen-chatglm-6b-ft/checkpoint-50/

                           PRE global_step50/
2023-05-08 15:14:49        870 config.json
2023-05-08 15:14:49       4276 configuration_chatglm.py
2023-05-08 15:14:49        142 generation_config.json
2023-05-08 15:14:49    2706249 ice_text.model
2023-05-08 15:14:49         13 latest
2023-05-08 15:14:49      57620 modeling_chatglm.py
2023-05-08 15:14:49 12346621179 pytorch_model-00001-of-00002.bin
2023-05-08 15:14:49 12346585635 pytorch_model-00002-of-00002.bin
2023-05-08 15:14:49      33416 pytorch_model.bin.index.json
2023-05-08 15:14:49      15054 quantization.py
2023-05-08 15:14:49      14583 rng_state_0.pth
2023-05-08 15:14:49      14583 rng_state_1.pth
2023-05-08 15:14:49      14583 rng_state_2.pth
2023-05-08 15:14:49      14583 rng_state_3.pth
2023-05-08 15:14:49      14583 rng_state_4.pth
2023-05-08 15:14:49      14583 rng_state_5.pth
2023-05-08 15:14:49      14583 rng_state_6.pth
2023-05-08 15:14:49      14583 rng_state_7.pth
2023-05-08 15:14:49        125 special_tokens_map.jso

In [ ]:
##for local test only
!cd ./ChatGLM-6B/ptuning/ && bash ./ds_train_finetune.sh

In [153]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/llm/

                           PRE chatglm/
                           PRE model/
                           PRE models/
